In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

## Data Ingestion

In [5]:
from langchain_community.document_loaders import TextLoader

In [8]:
loader = TextLoader(r'/Users/atharvabot7/Downloads/MLOPS-Tutorials/LLMOps-Project/Agentic_AI.txt', encoding='utf8')

In [9]:
documents = loader.load()

In [10]:
documents

[Document(metadata={'source': '/Users/atharvabot7/Downloads/MLOPS-Tutorials/LLMOps-Project/Agentic_AI.txt'}, page_content='Agentic AI represents the next strategic inflection point in artificial intelligence—moving systems from passive responders to proactive, goal-driven entities capable of taking initiative. Unlike traditional models that wait for explicit prompts, agentic AI systems can reason about tasks, plan multi-step actions, and autonomously decide which tools or information sources to use. This shift unlocks far more flexible automation, enabling AI systems to function less like calculators and more like adaptive digital operators.\n\nThe true power of agentic AI lies in its ability to orchestrate complex workflows. An agentic system can break down a high-level objective into smaller tasks, gather the necessary data, call external APIs, or trigger downstream services—while continuously evaluating its own progress. These capabilities make agentic AI especially useful in domain

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 20)

In [15]:
text_chunks = text_splitter.split_documents(documents)

In [16]:
text_chunks

[Document(metadata={'source': '/Users/atharvabot7/Downloads/MLOPS-Tutorials/LLMOps-Project/Agentic_AI.txt'}, page_content='Agentic AI represents the next strategic inflection point in artificial intelligence—moving systems from passive responders to proactive, goal-driven entities capable of taking initiative. Unlike'),
 Document(metadata={'source': '/Users/atharvabot7/Downloads/MLOPS-Tutorials/LLMOps-Project/Agentic_AI.txt'}, page_content='initiative. Unlike traditional models that wait for explicit prompts, agentic AI systems can reason about tasks, plan multi-step actions, and autonomously decide which tools or information sources to'),
 Document(metadata={'source': '/Users/atharvabot7/Downloads/MLOPS-Tutorials/LLMOps-Project/Agentic_AI.txt'}, page_content='sources to use. This shift unlocks far more flexible automation, enabling AI systems to function less like calculators and more like adaptive digital operators.'),
 Document(metadata={'source': '/Users/atharvabot7/Downloads/MLOPS